In [64]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from copy import deepcopy

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
torch.manual_seed(1000)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [9]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./.data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1, shuffle=True, **kwargs)

100.1%

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw


113.5%

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw


100.4%

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw


180.4%

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw
Processing...
Done!


In [10]:
init_num_workers = 4
models = [Net().to(device) for i in range(init_num_workers)]
optimizers = [optim.SGD(model.parameters(), lr=0.0001, momentum=0.1) for model in models]

In [43]:
def broadcast_params(model_list, root_rank=0):
    indices = [i for i in range(len(model_list))]
    indices.remove(root_rank)
    for i in indices:
        print(i)
        model_list[i].load_state_dict(model_list[root_rank].state_dict())

In [105]:
def if_all_params_require(papram_tuple):
    requires_grad = [p.requires_grad for p in papram_tuple]
    if True in requires_grad and False not in requires_grad:
        return True
    if False in requires_grad and True not in requires_grad:
        return False
    if True in requires_grad and False in requires_grad:
        print('Warning! Parameters has different gradient setting')
    return False

In [111]:
def allreduce_params(model_list, ops=sum):
    param_list = [m.parameters() for m in model_list]
    for param_tuple in zip(*param_list):
        if if_all_params_require(param_tuple):
            grad_temp = ops([p.grad.data for p in param_tuple])
            for p in param_tuple:
                p.grad.data = grad_temp

In [ ]:
# An example for using allreduce_params
broadcast_params(models)
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    for model, optimizer in zip(models, optimizers):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
    allreduce_params(models)
    for optimizer in optimizers:
        optimizer.step()